In [ ]:
from XrayTo3DShape import AttentionUnet, get_model, get_transform_from_model_name, get_dataset, VolumeAsInputExperiment
import torch
from torch.utils.data import DataLoader
import ipyvolume as ipv
import matplotlib.pyplot as plt

In [2]:
# load model checkpoint
model_path = '../runs/2d-3d-benchmark/yiw2kgep/checkpoints/epoch=60-step=2867-val_dice=0.85.ckpt'
checkpoint = torch.load(model_path)
for key in list(checkpoint["state_dict"].keys()):
    # model.layer1.conv1 -> layer1.conv1
    if str(key).startswith("model."):
        modified_key = str(key)[len("model.") :]
        value = checkpoint["state_dict"].pop(key)
        checkpoint["state_dict"][modified_key] = value
if "loss_function.pos_weight" in checkpoint["state_dict"]:
    checkpoint["state_dict"].pop("loss_function.pos_weight")

In [3]:
model_architecture = get_model(model_name=AttentionUnet.__name__, image_size=128)
model_architecture.load_state_dict(checkpoint["state_dict"], strict=False)


<All keys matched successfully>

In [4]:
test_datapath = '../test_data/totalsegmentation_hips_test.csv'

In [5]:
test_transform = get_transform_from_model_name(
    AttentionUnet.__name__, image_size=128, resolution=2.25
)

test_loader = DataLoader(
    get_dataset(test_datapath, transforms=test_transform),
    batch_size=1,
    num_workers=1,
    shuffle=False,
    drop_last=False,
)

In [ ]:
expt = VolumeAsInputExperiment(model_architecture)
for idx, item in enumerate(test_loader):
    input,seg = expt.get_input_output_from_batch(item)
    out = expt.predict_step(item,idx)
    pred, gt = out['pred'], out['gt']
    
    print(input[0].shape)


In [10]:
fig = ipv.figure()
vol = ipv.volshow(pred[0,0])
ipv.show()

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…